# Introduction

This polyglot will connect to the AI Toolkit and run a .net sample to run the code locally.

In [3]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.2"

Installed Packages Azure.AI.OpenAI, 2.0.0-beta.2

In [5]:
// OverridePolicy.cs
using Azure.Core.Pipeline;
using Azure.Core;

internal partial class OverrideRequestUriPolicy(Uri overrideUri)
    : HttpPipelineSynchronousPolicy
{
    private readonly Uri _overrideUri = overrideUri;

    public override void OnSendingRequest(HttpMessage message)
    {
        message.Request.Uri.Reset(_overrideUri);
    }
}

In [6]:
using Azure.AI.OpenAI;

// Parameters for the clientOptions
string deploymentModel = "Phi-3-mini-4k-directml-int4-awq-block-128-onnx";
string systemMessage = "You are a helpful assistant. Be brief and succinct.";
string prompt = "What is the golden ratio?";



// Running the model based on the parameters

Uri localhostUri = new("http://localhost:5272/v1/chat/completions");

OpenAIClientOptions clientOptions = new();
clientOptions.AddPolicy(
    new OverrideRequestUriPolicy(localhostUri),
    Azure.Core.HttpPipelinePosition.BeforeTransport);
OpenAIClient client = new(openAIApiKey: "unused", clientOptions);

ChatCompletionsOptions options = new()
{
    DeploymentName = deploymentModel,
    Messages =
    {
        new ChatRequestSystemMessage(systemMessage),
        new ChatRequestUserMessage(prompt),
    }
};

StreamingResponse<StreamingChatCompletionsUpdate> streamingChatResponse
    = await client.GetChatCompletionsStreamingAsync(options);

await foreach (StreamingChatCompletionsUpdate chatChunk in streamingChatResponse)
{
    Console.Write(chatChunk.ContentUpdate);
}

Error: (14,1): error CS0246: The type or namespace name 'OpenAIClientOptions' could not be found (are you missing a using directive or an assembly reference?)
(18,1): error CS0246: The type or namespace name 'OpenAIClient' could not be found (are you missing a using directive or an assembly reference?)
(20,1): error CS0246: The type or namespace name 'ChatCompletionsOptions' could not be found (are you missing a using directive or an assembly reference?)
(30,1): error CS0246: The type or namespace name 'StreamingResponse<>' could not be found (are you missing a using directive or an assembly reference?)
(30,19): error CS0246: The type or namespace name 'StreamingChatCompletionsUpdate' could not be found (are you missing a using directive or an assembly reference?)
(25,13): error CS0246: The type or namespace name 'ChatRequestSystemMessage' could not be found (are you missing a using directive or an assembly reference?)
(26,13): error CS0246: The type or namespace name 'ChatReques

> Note: Need to ensure the web server for Web AP Toolkit is working
```powershell
$body = Get-Content -Raw -Path sample1.json
Invoke-WebRequest -Uri http://127.0.0.1:5272/v1/chat/completions -Method Post -ContentType "application/json" -Body $body
```